In [1]:
import re    
import nltk 
from nltk.corpus import stopwords
import string 
import warnings 
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt  
import lightgbm as lgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn import tree
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV,StratifiedKFold,RandomizedSearchCV

In [2]:
xtrain = pd.read_csv('train.csv')
xtrain

,target,long,Tags,with_url,text
0,1,69,0,0,Our Deeds are the Reason of this #earthquake M...
1,1,38,0,0,Forest fire near La Ronge Sask. Canada
2,1,133,0,0,All residents asked to 'shelter in place' are ...
3,1,65,0,0,"13,000 people receive #wildfires evacuation or..."
4,1,88,0,0,Just got sent this photo from Ruby #Alaska as ...
...,...,...,...,...,...
7608,1,83,0,1,Two giant cranes holding a bridge collapse int...
7609,1,125,2,0,@aria_ahrary @TheTawniest The out of control w...
7610,1,65,0,1,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...
7611,1,137,0,0,Police investigating after an e-bike collided ...


In [3]:
def clean_text(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [4]:
xtrain['text'] = xtrain['text'].apply(lambda x: clean_text(x))

In [5]:
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
xtrain['text'] = xtrain['text'].apply(lambda x: tokenizer.tokenize(x))

In [6]:
def remove_stopwords(text):
    words = []
    for w in text:
        if w not in stopwords.words('english'):
            words.append(w)
    #words = [w for w in text if w not in stopwords.words('english')]
    return words

xtrain['text'] = xtrain['text'].apply(lambda x: remove_stopwords(x))

In [7]:
def combine_text(list_of_text):
    combined_text = ' '.join(list_of_text)
    return combined_text

In [8]:
xtrain['text'] = xtrain['text'].apply(lambda x: combine_text(x))

In [9]:
tokenized_tweet = xtrain.text.apply(lambda x: x.split())

In [10]:
def word_vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0
    for word in tokens:
        try:
            vec += model_glovec[word].reshape((1, size))
            count += 1.
        except KeyError:  # handling the case where the token is not in vocabulary
            continue
    if count != 0:
        vec /= count
    return vec

In [11]:
model_glovec = {}
with open("glove.twitter.27B.200d.txt", 'r', encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        model_glovec[word] = vector

In [12]:
wordvec_arrays = np.zeros((len(tokenized_tweet), 200)) 
for i in range(len(tokenized_tweet)):
    wordvec_arrays[i,:] = word_vector(tokenized_tweet[i], 200)
wordvec_df = pd.DataFrame(wordvec_arrays)
wordvec_df.shape

(7613, 200)

In [13]:
len(tokenized_tweet)

7613

In [14]:
xtest = pd.read_csv('dftestLSTM.csv')
xtest =xtest.drop(columns='Unnamed: 0')
xtest.text = xtest.text.apply(lambda x: clean_text(x))
xtest.text = xtest.text.apply(lambda x: tokenizer.tokenize(x))
xtest.text = xtest.text.apply(lambda x: remove_stopwords(x))
xtest.text = xtest.text.apply(lambda x: combine_text(x))
tokenized_test = xtest.text.apply(lambda x: x.split()) # tokenizing 
wordvec_test = np.zeros((len(tokenized_test), 200)) 
for i in range(len(tokenized_test)):
    wordvec_test[i,:] = word_vector(tokenized_test[i], 200)
wordvect_df = pd.DataFrame(wordvec_test)
wordvect_df.shape

(3263, 200)

In [16]:
xtrain_gv = wordvec_df
xtest_gv = wordvect_df
ytrain= xtrain.target


In [17]:
model = lgb.LGBMClassifier(max_depth=25, n_estimators=1500, nthread= 4, eta=0.1).fit(xtrain_gv, ytrain) 
prediction = model.predict(xtest_gv)

In [1]:
ids=[]
with open("sample_submission.csv", 'r') as f:
    for line in f:
        vals = line.split(',')
        if vals[0]!='id':
            ids.append(int(vals[0]))

In [22]:
en = open("submission.csv", 'w')
n=0
en.write("id,target\n")
for i in ids:
    en.write(str(i) +','+ str(prediction[n])+'\n')
    
    n+=1
en.close()